In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
# write a function to read files to a dictionary
def read_file(file_path, nrows = 1000):
    datadict = {"CustomerID":[],"ProductID":[],"Rating":[]}; # dictionary holder, no values
    file = open(file_path,"r"); # open file for reading
    count =1;
    for line in file:
        if count>nrows:
            break;
        if ":" in line:
            movie_id = line[:-2];
            movie_id = int(movie_id);
        else:
            customer_id,rating,date = line.split(",");
            datadict["CustomerID"].append(customer_id) ;
            datadict["ProductID"].append(movie_id);
            datadict["Rating"].append(rating);
            # exclude date because we do not need it for prediction
        count +=1;
    file.close(); #close file after reading
    return pd.DataFrame(datadict); 

In [ ]:
# Read Data from the file
nRow = 50000;
filepath = "/kaggle/input/netflix-prize-data/combined_data_1.txt"
df1 = read_file(filepath, nrows = nRow);
filepath = "/kaggle/input/netflix-prize-data/combined_data_2.txt"
df2 = read_file(filepath, nrows = nRow);
filepath = "/kaggle/input/netflix-prize-data/combined_data_3.txt"
df3 = read_file(filepath, nrows = nRow);
filepath = "/kaggle/input/netflix-prize-data/combined_data_4.txt"
df4 = read_file(filepath, nrows = nRow);
# merge data to make a user-product data
UserProductData = df1.append(df2);
UserProductData=UserProductData.append(df3);
UserProductData.append(df4)
print(UserProductData.head(5));
UserProductData["Rating"] = UserProductData["Rating"].astype("float"); # convert to float because it is used in computation
# Read movie data from which to predict movie
movieTitles = pd.read_csv("/kaggle/input/netflix-prize-data/movie_titles.csv",header=None, encoding = "ISO-8859-1", names=["ProductID","Year", "MovieTitle"]);
movieTitles.head(5)

In [ ]:
# Load libraries from python surprice package
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate

In [ ]:
# create objects of Reader and SVD classes
reader = Reader();
svd = SVD();

In [ ]:
# prepare the trainig data set in the order of "item, user, rating"
train_data = Dataset.load_from_df(UserProductData[["ProductID","CustomerID","Rating"]],reader);

# validate the svd model with cross_validate
cross_validate(svd,train_data,measures = ["RMSE","MAE"],cv = 5,verbose = True);

# build model on the entire data set
train_set = train_data.build_full_trainset();

# fit model on train set
svd.fit(train_set);




In [ ]:
# use the model for recommendation for a userID, say userID = UserProductData.iloc[0,0]
userID = 785314;#UserProductData.iloc[0,0]; # example of a customer
movieTitles["EstimateRank"] = movieTitles["ProductID"].apply(lambda x: svd.predict(userID,x).est);
movieTitles=movieTitles.sort_values(by=["EstimateRank"], ascending = False)
movieTitles.head(10)